In [11]:
from IPython.display import display
import mne
from components.dataset import Dataset_Builder
import pickle
import os
import pandas as pd
import numpy as np
dataset_path = "data"
dataset = Dataset_Builder(dataset_path=dataset_path, EEG_ext='.csv', EEG_folder='EEG_csv')\
            .with_sampling_rate(125)\
            .with_marker(start_minute=1, stop_minute=3, segment_second=6)\
            .build()

In [12]:
def save(data, filename):
    with open(f'data/EEG_numpy_0_5/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
def load(filename):
    with open(f'data/EEG_numpy_0_5/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

In [14]:
def to_raw(df):
    ch_names = list(df.columns)
    ch_types = ['eeg'] * len(ch_names)
    # https://mne.tools/stable/generated/mne.create_info.html
    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=125)

    df = df.T  #mne looks at the tranpose() format
    df *= 1e-6  #convert from uVolts to Volts (mne assumes Volts data)

    raw = mne.io.RawArray(df,info=info,verbose='CRITICAL')
    raw.set_montage('standard_1020')
    return raw

In [15]:
def get_markers():
    sampling_rate = 125 #Hz
    # 15/60 = 0.25
    step_minutes = np.arange(0, 0.1,0.25)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers
markers = get_markers()
markers.shape
markers

step_minutes=array([0.])


array([[0, 0, 1]])

In [16]:
pd.read_csv(f)

,Index,Fp1,Fp2,F3,F4,F7,F8,C3,C4,T3,T4,T5,T6,P3,P4,O1,O2
0,0,9845.385,9995.700,7552.364,3283.963,9159.119,8703.456,6909.818,4040.748,9158.247,10311.799,11623.019,3934.958,2306.432,4978.672,4585.438,3508.330
1,1,9857.634,10007.301,7526.235,3246.703,9173.401,8714.968,6925.129,4054.629,9102.882,10252.991,11574.314,3882.364,2257.772,4929.141,4543.953,3467.158
2,2,9885.081,10036.716,7525.653,3231.571,9197.922,8744.427,6951.125,4080.266,9163.188,10318.549,11637.123,3943.362,2320.402,4988.842,4596.301,3516.801
3,3,9836.868,9987.072,7569.351,3310.316,9153.777,8694.337,6900.095,4029.349,9124.295,10285.065,11593.604,3900.938,2274.513,4947.291,4559.622,3481.038
4,4,9898.403,10050.395,7512.779,3217.511,9214.395,8759.626,6962.255,4092.090,9133.861,10279.276,11611.105,3918.216,2289.936,4959.964,4575.089,3495.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59563,59563,10491.797,10147.960,7194.870,1713.909,11162.171,9360.844,6527.693,6298.208,9028.316,10938.541,12913.452,3129.222,752.829,3675.454,3978.789,2553.754
59564,59564,10453.039,10106.788,7235.148,1785.949,11124.888,9318.487,6488.846,6258.600,8973.130,10883.266,12857.551,3072.963,696.950,3623.598,3939.361,2513.230
59565,59565,10517.792,10172.883,7165.746,1671.843,11190.133,9381.898,6549.307,6318.749,9007.463,10922.560,12893.760,3111.899,730.790,3659.427,3962.942,2541.013
59566,59566,10461.712,10116.221,7240.646,1788.430,11135.975,9327.830,6488.823,6263.227,9015.106,10928.014,12900.376,3121.734,735.261,3671.341,3975.548,2549.306


In [21]:
for f in dataset.files:
    _,name = os.path.split(f)
    name,ext = os.path.splitext(name)
    df = pd.read_csv(f, index_col='Index')
    # print(df.shape)
    raw = to_raw(df)
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=5*60, baseline=None, preload=True)
    data = epochs.get_data()[0]
    print(f, data[:, :-1].shape)
    assert data[:, :-1].shape == (16,37500)
    save(data[:, :-1], name)
    # break
    
    # break
    # df = df.loc[0.5*125*60: (5.5*125*60)-1]
    # df = df.T
    # df = df * 1e-6
    # data = df.to_numpy()
    # print(name, data.shape)
    # save(data, name)
    # print(f, df.shape, df.shape[1]/125)
    # break

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 37501 original time points ...
0 bad epochs dropped
data/EEG_csv/001.csv (16, 37500)
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 37501 original time points ...
0 bad epochs dropped
data/EEG_csv/002.csv (16, 37500)
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 37501 original time points ...
0 bad epochs dropped
data/EEG_csv/003.csv (16, 37500)
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 37501 original time points ...
0 bad epochs dropped
data/EEG_csv/004.csv (16, 37500)
Not setting metadata
1 matching events found
No base